<img src=".\images\Logo.png" width=150 align="left" /> <img src=".\images\Logo2.jpg" width=450 align="right" />


# <center><font color= #1e8449 > <b>CU18_Infraestructuras_eventos</font></center>

<font color='steelblue'><b>Citizenlab Data Science Methodology > II - Data Processing Domain </font>
***
> # <font color='steelblue'> <b>05.- Data Collection</font>

<font color='steelblue'>Data Collection is the process to obtain and generate (if required) necessary data to model the problem.</font>

### 15. Agregar datos meteorológicos por distritos y año

* Estimar variables en distritos
* Agrupar promedios anuales de los distritos en las variables meteorológicas
* Eliminando variables que no se usarán en modelos







<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Settings" data-toc-modified-id="Settings-1"><font color="green">Settings</font></a></span></li><li><span><a href="#Data-Load" data-toc-modified-id="Data-Load-2"><font color="green">Data Load</font></a></span></li><li><span><a href="#ETL-Processes" data-toc-modified-id="ETL-Processes-3">ETL Processes</a></span><ul class="toc-item"><li><span><a href="#Import-data-from:-CSV,-Excel,-Tab,-JSON,-SQL,-and-Parquet-files" data-toc-modified-id="Import-data-from:-CSV,-Excel,-Tab,-JSON,-SQL,-and-Parquet-files-3.1">Import data from: CSV, Excel, Tab, JSON, SQL, and Parquet files</a></span></li></ul></li><li><span><a href="#Synthetic-Data-Generation" data-toc-modified-id="Synthetic-Data-Generation-4">Synthetic Data Generation</a></span></li><li><span><a href="#Fake-Data-Generation" data-toc-modified-id="Fake-Data-Generation-5">Fake Data Generation</a></span></li><li><span><a href="#Open-Data" data-toc-modified-id="Open-Data-6">Open Data</a></span></li><li><span><a href="#Data-Save" data-toc-modified-id="Data-Save-7"><font color="green">Data Save</font></a></span></li><li><span><a href="#Main-Conclusions" data-toc-modified-id="Main-Conclusions-8"><font color=" #2874a6 ">Main Conclusions</font></a></span></li><li><span><a href="#Main-Actions" data-toc-modified-id="Main-Actions-9"><font color=" #2874a6 ">Main Actions</font></a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Acciones-done" data-toc-modified-id="Acciones-done-9.0.1"><font color=" #2874a6 ">Acciones done</font></a></span></li><li><span><a href="#Acctions-to-perform" data-toc-modified-id="Acctions-to-perform-9.0.2"><font color=" #2874a6 ">Acctions to perform</font></a></span></li></ul></li></ul></li></ul></div>

## <font color='green'>Settings</font>

### Packages to use

- {tcltk} para selección interactiva de archivos locales
- {sf} para trabajar con georeferenciación
- {gstat} para cálculos geoestadísticos
- {readr} para leer y escribir archivos csv
- {dplyr} para explorar datos
- {stringr} para manipulación de cadenas de caracteres
- {tidyr} para quitar perdidos


In [1]:
library(readr)
library(dplyr)
library(sf)
library(gstat)
library(stringr)
library(tidyr)



Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Linking to GEOS 3.10.2, GDAL 3.4.2, PROJ 8.2.1; sf_use_s2() is TRUE



### Paths

In [2]:
iPath <- "Data/Input/"
oPath <- "Data/Output/"

## <font color='green'>Data Load</font>

If there are more than one input file, make as many sections as files to import.

Instrucciones
- Los ficheros de entrada del proceso están siempre en Data/Input/.   
- Si hay más de un fichero de entrada, se crean tantos objetos iFile_xx y file_data_xx como ficheros de entrada (xx número correlativo con dos dígitos, rellenar con ceros a la izquierda)

1. Datos meteorológicos de estaciones

<font color='tomato'><b> OPCION A:</b> Seleccionar fichero en ventana para mayor comodidad</font> 

Data load using the {tcltk} package. Ucomment the line if not using this option

In [3]:
# file_data_01 <- tcltk::tk_choose.files(multi = FALSE)

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>



In [4]:
iFile_01 <- "CU_18_05_14_aemet.csv"
file_data_01 <- paste0(iPath, iFile_01)

if(file.exists(file_data_01)){
    cat("Se leerán datos del archivo: ", file_data_01)
} else{
    warning("Cuidado: el archivo no existe.")
}


Se leer<U+00E1>n datos del archivo:  Data/Input/CU_18_05_14_aemet.csv

#### Data file to dataframe

Usar la función adecuada según el formato de entrada (xlsx, csv, json, ...)

In [5]:
data_01 <- read_csv(file_data_01)


Rows: 4547 Columns: 22
-- Column specification --------------------------------------------------------
Delimiter: ","
chr  (10): indicativo, nombre, provincia, prec, horatmin, horatmax, dir, hor...
dbl  (11): altitud, tmed, tmin, tmax, velmedia, racha, sol, presMax, presMin...
date  (1): fecha

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


Estructura de  los datos:

In [6]:
glimpse(data_01)

Rows: 4,547
Columns: 22
$ fecha       <date> 2019-01-01, 2019-01-02, 2019-01-03, 2019-01-04, 2019-01-0~
$ indicativo  <chr> "2462", "2462", "2462", "2462", "2462", "2462", "2462", "2~
$ nombre      <chr> "PUERTO DE NAVACERRADA", "PUERTO DE NAVACERRADA", "PUERTO ~
$ provincia   <chr> "MADRID", "MADRID", "MADRID", "MADRID", "MADRID", "MADRID"~
$ altitud     <dbl> 1894, 1894, 1894, 1894, 1894, 1894, 1894, 1894, 1894, 1894~
$ tmed        <dbl> 6.4, 5.2, 7.6, 6.0, 7.2, 4.6, 4.1, 6.2, -0.8, -4.1, -5.2, ~
$ prec        <chr> "0,0", "0,0", "0,0", "0,0", "0,0", "0,0", "0,0", "0,0", "0~
$ tmin        <dbl> 1.6, -0.8, 2.7, 1.7, 1.8, 0.9, 1.2, 1.4, -3.5, -7.5, -10.3~
$ horatmin    <chr> "23:40", "08:00", "23:59", "01:30", "18:00", "07:10", "Var~
$ tmax        <dbl> 11.3, 11.3, 12.6, 10.2, 12.5, 8.2, 7.0, 10.9, 2.0, -0.7, -~
$ horatmax    <chr> "12:40", "12:40", "12:00", "12:10", "12:50", "11:40", "13:~
$ dir         <chr> "36", "04", "15", "15", "03", "35", "36", "02", "36", "36"~
$ velmedia    <d

Muestra de datos:

In [7]:
slice_head(data_01, n = 5)

fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,...,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin,X,Y
<date>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,...,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>
2019-01-01,2462,PUERTO DE NAVACERRADA,MADRID,1894,6.4,"0,0",1.6,23:40,11.3,...,3.3,8.9,23:59,7.9,823.1,10,821.7,05,-4.010556,40.79306
2019-01-02,2462,PUERTO DE NAVACERRADA,MADRID,1894,5.2,"0,0",-0.8,08:00,11.3,...,2.8,11.4,17:10,7.9,822.0,00,818.8,24,-4.010556,40.79306
2019-01-03,2462,PUERTO DE NAVACERRADA,MADRID,1894,7.6,"0,0",2.7,23:59,12.6,...,1.9,8.9,04:00,7.3,819.5,10,818.0,05,-4.010556,40.79306
2019-01-04,2462,PUERTO DE NAVACERRADA,MADRID,1894,6.0,"0,0",1.7,01:30,10.2,...,1.4,8.3,00:40,8.0,822.7,23,818.5,05,-4.010556,40.79306
2019-01-05,2462,PUERTO DE NAVACERRADA,MADRID,1894,7.2,"0,0",1.8,18:00,12.5,...,3.1,8.3,06:50,8.2,824.3,10,822.4,Varias,-4.010556,40.79306


2. Datos de contornos distritos

<font color='tomato'><b> OPCION A:</b> Seleccionar fichero en ventana para mayor comodidad</font> 

Data load using the {tcltk} package. Ucomment the line if not using this option

In [8]:
# file_data_02 <- tcltk::tk_choose.files(multi = FALSE)

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>



In [9]:
iFile_02 <- "CU_18_05_03_distritos_geo.json"
file_data_02 <- paste0(iPath, iFile_02)

if(file.exists(file_data_02)){
    cat("Se leerán datos del archivo: ", file_data_02)
} else{
    warning("Cuidado: el archivo no existe.")
}


Se leer<U+00E1>n datos del archivo:  Data/Input/CU_18_05_03_distritos_geo.json

#### Data file to dataframe

Usar la función adecuada según el formato de entrada (xlsx, csv, json, ...)

In [10]:
data_02 <- st_read(file_data_02)


Reading layer `CU_18_05_03_distritos_geo' from data source 
  `/Users/emilio.lcano/academico/gh_repos/__transferencia/citizenlab/CitizenLab-Research-and-Development/casos_urjc/notebooks/II_data_processing/18_infraestructuras/Data/Input/CU_18_05_03_distritos_geo.json' 
  using driver `GeoJSON'
Simple feature collection with 247 features and 2 fields
Geometry type: GEOMETRY
Dimension:     XY
Bounding box:  xmin: -4.579006 ymin: 39.8848 xmax: -3.052983 ymax: 41.16584
Geodetic CRS:  WGS 84


Estructura de  los datos:

In [11]:
glimpse(data_02)

Rows: 247
Columns: 3
$ CMUN     <chr> "001", "002", "003", "004", "005", "005", "005", "005", "005"~
$ CDIS     <chr> "01", "01", "01", "01", "01", "02", "03", "04", "05", "01", "~
$ geometry <POLYGON [arc_degree]> POLYGON ((-3.64502 41.12129..., POLYGON ((-3~


Muestra de datos:

In [12]:
data_02 |> tibble() |> slice_head(n = 5)

CMUN,CDIS,geometry
<chr>,<chr>,<POLYGON [arc_degree]>
001,01,POLYGON ((-3.64502 41.12129...
002,01,"POLYGON ((-3.503032 40.526,..."
003,01,POLYGON ((-3.808664 40.8921...
004,01,POLYGON ((-4.00197 40.25642...
005,01,POLYGON ((-3.361691 40.4762...


## ETL Processes

### Import data from: CSV, Excel, Tab, JSON, SQL, and Parquet files

Se han importado en el apartado Data Load anterior:

* Datos diarios meteorológicos por estación
* Contornos de distritos censales

Incluir apartados si procede para: Extracción de datos (select, filter), Transformación de datos, (mutate, joins, ...). Si es necesario tratar datos perdidos, indicarlo también en NB 09.2

Si no aplica: Estos datos no requieren tareas de este tipo.

#### Data transform

* Transformar precipitación a numérica
* Ip es inapreciable, se sustituye por 0

* Corregir geometrías distritos y eliminar vacíos

In [13]:
tdata_01 <- data_01 |>
    mutate(prec = str_replace(prec, "Ip", "0"),
        prec = as.numeric(str_replace_all(prec, ",", ".")))

In [14]:
tdata_02 <- data_02 |> st_make_valid()
tdata_02 <- tdata_02 |> filter(!st_is_empty(tdata_02))

#### Data extract

* Seleccionar variables temperatura, precipitaciones y presión
* Agrupar por año

In [15]:
tdata_01_grouped <- tdata_01 |>
    select(nombre, tmed, prec, velmedia, presMax, X, Y) |>
    group_by(nombre, X, Y) |>
    summarise(across(everything(), ~mean(.x, na.rm = TRUE)))

`summarise()` has grouped output by 'nombre', 'X'. You can override using the
`.groups` argument.


In [16]:
glimpse(tdata_01_grouped)

Rows: 13
Columns: 7
Groups: nombre, X [13]
$ nombre   <chr> "ARANJUEZ", "BUITRAGO DEL LOZOYA", "COLMENAR VIEJO", "GETAFE"~
$ X        <dbl> -3.546111, -3.613611, -3.765000, -3.722222, -3.555556, -3.724~
$ Y        <dbl> 40.06722, 41.00694, 40.69611, 40.29944, 40.46667, 40.45167, 4~
$ tmed     <dbl> 15.584874, 12.800000, 14.219178, 16.398356, 15.673973, 14.484~
$ prec     <dbl> 0.7657224, 1.3893023, 1.1323288, 0.9484932, 0.9210959, 1.0198~
$ velmedia <dbl> 1.822830, 2.017447, 2.549589, 3.575890, 3.466301, 2.097973, 3~
$ presMax  <dbl> NaN, 905.8165, 905.7819, 948.3597, 952.3433, NaN, 940.5129, 9~


In [17]:
tdata_01_grouped

nombre,X,Y,tmed,prec,velmedia,presMax
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ARANJUEZ,-3.546111,40.06722,15.584874,0.7657224,1.822830,NaN
BUITRAGO DEL LOZOYA,-3.613611,41.00694,12.800000,1.3893023,2.017447,905.8165
COLMENAR VIEJO,-3.765000,40.69611,14.219178,1.1323288,2.549589,905.7819
GETAFE,-3.722222,40.29944,16.398356,0.9484932,3.575890,948.3597
MADRID AEROPUERTO,-3.555556,40.46667,15.673973,0.9210959,3.466301,952.3433
"MADRID, CIUDAD UNIVERSITARIA",-3.724167,40.45167,14.484516,1.0198347,2.097973,NaN
"MADRID, CUATRO VIENTOS",-3.786111,40.37556,16.144658,1.0112329,3.284658,940.5129
"MADRID, RETIRO",-3.678056,40.41194,16.143288,1.0726027,2.040274,942.5658
PUERTO ALTO DEL LE<U+00D3>N,-4.141944,40.70639,9.677966,1.3355932,5.075424,850.7794


#### Data transform

* Convertir a objeto `sf` para poder hacer las estimaciones

In [18]:
tdata_01_grouped_sf <- st_as_sf(tdata_01_grouped, coords = c("X", "Y"), crs = 4326)

#### Data transform

* Estimar valores medios de cada distrito con los datos puntuales
* Unir a la tabla

In [19]:


tmed <- idw(tmed ~ 1, 
          locations = tdata_01_grouped_sf |> drop_na(tmed), 
          newdata = tdata_02)

prec <- idw(prec ~ 1, 
          locations = tdata_01_grouped_sf |> drop_na(prec), 
          newdata = tdata_02)

velmedia <- idw(velmedia ~ 1, 
          locations = tdata_01_grouped_sf |> drop_na(velmedia), 
          newdata = tdata_02)
presMax <- idw(presMax ~ 1, 
          locations = tdata_01_grouped_sf |> drop_na(presMax), 
          newdata = tdata_02)


[inverse distance weighted interpolation]
[inverse distance weighted interpolation]
[inverse distance weighted interpolation]
[inverse distance weighted interpolation]


In [20]:
tdata_02_meteo <- tdata_02 |>
    mutate(tmed = tmed$var1.pred,
            prec = prec$var1.pred,
            velmedia = velmedia$var1.pred,
            presMax = presMax$var1.pred) |>
    st_drop_geometry()

In [21]:
glimpse(tdata_02_meteo)

Rows: 246
Columns: 6
$ CMUN     <chr> "001", "002", "003", "004", "005", "005", "005", "005", "005"~
$ CDIS     <chr> "01", "01", "01", "01", "01", "02", "03", "04", "05", "01", "~
$ tmed     <dbl> 11.47600, 15.31806, 12.00935, 15.13839, 15.27439, 15.31156, 1~
$ prec     <dbl> 3.6992270, 0.8713863, 2.0387942, 1.0548851, 0.8788330, 0.8511~
$ velmedia <dbl> 2.090249, 3.352072, 2.986153, 2.919731, 3.331738, 3.372923, 3~
$ presMax  <dbl> 904.2719, 946.6977, 880.7764, 931.9199, 945.8412, 946.8914, 9~


In [22]:
tdata_02_meteo |> slice_head(n = 5)

CMUN,CDIS,tmed,prec,velmedia,presMax
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
001,01,11.47600,3.6992270,2.090249,904.2719
002,01,15.31806,0.8713863,3.352072,946.6977
003,01,12.00935,2.0387942,2.986153,880.7764
004,01,15.13839,1.0548851,2.919731,931.9199
005,01,15.27439,0.8788330,3.331738,945.8412


####

## Synthetic Data Generation

No aplica

## Fake Data Generation

No aplica

## Open Data

Los datos originales fueron descargados de fuentes abiertas:

* Meteorológicas de AEMET
* Geográficas (distritos) del INE

## <font color='green'>Data Save</font>

Este proceso, puede copiarse y repetirse en aquellas partes del notebbok que necesiten guardar datos.
Recuerde cambiar las cadenas añadida del fichero para diferenciarlas

<font color='tomato'> Identificamos los datos a guardar</font>

In [23]:
data_to_save <- tdata_02_meteo


<font color='tomato'>Estructura de nombre de archivos:</font>

* Código del caso de uso, por ejemplo "CU_04"
* Número del proceso que lo genera, por ejemplo "_05".
* Número de la tarea que lo genera, por ejemplo "_01"
* En caso de generarse varios ficheros en la misma tarea, llevarán _01 _02 ... después
* Nombre: identificativo de "properData", por ejemplo "_zonasgeo"
* Extensión del archivo

Ejemplo: "CU_04_05_01_01_zonasgeo.json, primer fichero que se genera en la tarea 01 del proceso 05 (Data Collection) para el caso de uso 04 (vacunas)

Importante mantener los guiones bajos antes de proceso, tarea, archivo y nombre

### Proceso 05

In [24]:
caso <- "CU_18"
proceso <- '_05'
tarea <- "_15"
archivo <- ""
proper <- "_distritos_meteo"
extension <- ".csv"

<font color='tomato'><b> OPCION A:</b> Uso del paquete "tcltk" para mayor comodidad</font>

* Buscar carpeta, escribir nombre de archivo SIN extensión (se especifica en el código)
* Especificar sufijo2 si es necesario
* Cambiar datos por datos_xx si es necesario

In [25]:
# file_save <- paste0(caso, proceso, tarea, tcltk::tkgetSaveFile(), proper, extension) 
# path_out <- paste0(oPath, file_save)
# write_csv(data_to_save, path_out)

# cat('File saved as: ')
# path_out

<font color='tomato'><b> OPCION B:</b> Especificar el nombre de archivo</font>

- Los ficheros de salida del proceso van siempre a Data/Output/.  

In [26]:
file_save <- paste0(caso, proceso, tarea, archivo, proper, extension) 
path_out <- paste0(oPath, file_save)
write_csv(data_to_save, path_out)

cat('File saved as: ')
path_out

File saved as: 

[1] "Data/Output/CU_18_05_15_distritos_meteo.csv"

#### Copia del fichero a Input

Si el archivo se va a usar en otros notebooks, copiar a la carpeta Input

In [27]:
path_in <- paste0(iPath, file_save)
file.copy(path_out, path_in, overwrite = TRUE)

[1] TRUE

## <font color=' #2874a6 '>Main Conclusions</font>

<font color='steelblue'>List and describe the general conclusions of the analysis carried out​.</font>

### Prerequisites

This working code needs the following conditions:

* For using the interactive selection of file, the {tcltk} package must be installed. It is not needed in production.
* The {sf}, {gstat}, {readr}, {dplyr}, {stringr} and {tidyr} packages must be installed. 
* The data paths `Data/Input` and `Data/Output` must exist (relative to the notebook path)

### Configuration Management

This notebook has been tested with the following versions of R and packages. It cannot be assured that later versions work in the same way:
* R 4.2.2
* tcltk 4.2.2
* sf 1.0.9
* gstat 2.1.0
* readr 2.1.3
* dplyr 1.0.10
* stringr 1.5.0
* tidyr 1.3.0

### Data structures

#### Objeto `data`

- Hay 246 filas con las variables:
    * CMUN
    * CDIS
    * tmed
    * prec
    * velmedia
    * presMax


In [28]:
colnames(tdata_02_meteo)

[1] "CMUN"     "CDIS"     "tmed"     "prec"     "velmedia" "presMax"

#### Observaciones generales sobre los datos

* Se han agregado los datos totales porque se tiene un año. Si se usan más años en los modelos, el agrupamiento debe ser de la granularidad de la que haya datos.

### Consideraciones para despliegue en piloto

* Si llegan datos reales se deberían descargar los datos de las fechas que coincidan

### Consideraciones para despliegue en producción

* Se deben crear los procesos ETL en producción necesarios para que los datos de entrada estén actualizados

## <font color=' #2874a6 '>Main Actions</font>

#### <font color=' #2874a6 '>Acciones done</font>

<font color='steelblue'>Indicate the actions that have been carried out in this process</font>

- Se han xxxx

#### <font color=' #2874a6 '>Acctions to perform</font>

<font color='steelblue'>Indicate the actions that must be carried out in subsequent processes</font>

- Se deben unir a los datos agregados de distritos del resto de variables

## <font color='RED'>CODE TO DEPLOY (PILOT)</font>

A continuación se incluirá el código que deba ser llevado a despliegue para producción, dado que se entiende efectúa operaciones necesarias sobre los datos en la ejecución del prototipo

<font color=' red '><b>Description</b></font>

- No hay nada que desplegar en el piloto, ya que estos datos son estáticos o en todo caso cambian con muy poca frecuencia, altamente improbable durante el proyecto.


<font color=' red '><b>CODE</b></font>

In [29]:
# incluir código